# Prepare dataset

In [1]:
# autoreload your package
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
from loguru import logger
from tqdm.auto import tqdm
# logger.remove()
# import sys
# logger.add(sys.stderr, level="INFO")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import pandas as pd
import json
from pathlib import Path

import lie_elicitation_prompts
from lie_elicitation_prompts.config import ExtractConfig
from lie_elicitation_prompts.helpers.scores import row_choice_ids
from lie_elicitation_prompts.prompts.prompt_loading import load_preproc_datasets, load_prompts

cfg = ExtractConfig(datasets=(
    '../lie_elicitation_prompts/prompts/templates/UKPLab-liar',
    "amazon_polarity", "glue:sst2", "super_glue:axg",
))
cfg
# lie_elicitation_prompts/prompts/templates/liar

ExtractConfig(datasets=('../lie_elicitation_prompts/prompts/templates/UKPLab-liar', 'amazon_polarity', 'glue:sst2', 'super_glue:axg'), datasets_ood=('imdb', 'super_glue:boolq'), model='failspy/Llama-3-8B-Instruct-abliterated', num_shots=2, max_tokens=776, max_examples=301, seed=42, repeats=3)

## Load text dataset

In [4]:

# # debug
# for ds_name in cfg.datasets:
#     print(ds_name)
#     o = load_prompts(ds_name, num_shots=1, N=2) 
#     o = list(tqdm(o))
#     # print(ds_name, o)
#     1/0
# pd.DataFrame(o)

In [5]:


# Ignore UserWarning category
# warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message="^The groups parameter is ignored by StratifiedShuffleSplit")


In [6]:
# # debug
# list(load_prompts(cfg.datasets[0], num_shots=0, N=2))

In [7]:


N = cfg.max_examples
ds_prompts = load_preproc_datasets(
    cfg.datasets,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    M=cfg.repeats,
)
ds_prompts

Generating train split: 0 examples [00:00, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.
2024-06-15 18:22:04.727 | INFO     | lie_elicitation_prompts.prompts.prompt_loading:load_prompts:118 - Extracting 2 variants of each prompt
2024-06-15 18:22:04.729 | INFO     | lie_elicitation_prompts.prompts.prompt_loading:load_prompts:133 - Using the following pseudo-labels: [0, 1]


DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
# ds_prompts_ood = load_preproc_datasets(
#     cfg.datasets_ood,
#     N=N,
#     seed=cfg.seed,
#     num_shots=cfg.num_shots,
# )
# ds_prompts_ood

In [ ]:
ds_prompts[1]

In [ ]:
# save

## Load tokenized dataset

- tokenize
- filter out truncated
- check which ones the model knows

In [ ]:
import os, torch
# os.environ['CUDA_VISIBLE_DEVICES'] = 'GPU-c4552741-f485-34ce-97fa-6c32983853af'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
# torch.cuda.get_device_name()

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    cfg.model,
    device_map="cuda:0",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(cfg.model)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

In [ ]:


ds_tokens = (
    ds_prompts.map(
        lambda x: {
            "formatted_chat": tokenizer.apply_chat_template(
                x["messages"], tokenize=False, add_generation_prompt=True
            )
        }
    )
    .map(
        lambda x: tokenizer(
            x["formatted_chat"],
            return_tensors="pt",
            max_length=cfg.max_tokens,
            padding="max_length",
            truncation=True,
        ),
        batched=True,
    )
    .map(lambda r: {"choice_ids": row_choice_ids(r, tokenizer)}, desc="choice_ids")
    .filter(lambda x: x["attention_mask"].sum() < cfg.max_tokens)
)
ds_tokens

In [ ]:

# def apply_prompt(messages):
#     o = tokenizer.apply_chat_template(
#                 messages, add_generation_prompt=True, return_tensors="pt",
#             max_length=cfg.max_tokens,
#             padding="max_length",
#             truncation=True,
#             return_dict=True,
#             )
#     return {k:v.squeeze() for k,v in o.items()}

# ds_tokens = (
#     ds_prompts.map(
#         lambda x: apply_prompt(x["messages"])
#     )
#     .map(lambda r: {"choice_ids": row_choice_ids(r, tokenizer)}, desc="choice_ids")
#     .filter(lambda x: x["attention_mask"].sum() < cfg.max_tokens)
# )
# ds_tokens

In [ ]:
print(len(ds_prompts), len(ds_tokens))

pd.Series(ds_prompts['ds_string']).value_counts(), pd.Series(ds_tokens['sys_instr_name']).value_counts()

In [ ]:
# tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)

ds_tokens[:4]['formatted_chat']

### Check model knowledge

In [ ]:
df_metadata = ds_tokens.select_columns(['ds_string', 'example_i', 'sys_instr_name', 'instructed_to_lie']).to_pandas().reset_index(names='my_ds_index')
df_metadata_truth = df_metadata.query('instructed_to_lie == False')
df_metadata_truth

# FIXME right now there is just one example of each, I guess I want a couple, hmm
df_metadata.query('instructed_to_lie == False').groupby(['ds_string', 'example_i'], as_index=False)['my_ds_index'].count()

In [ ]:
# ds_tokens_truthful = ds_tokens.select(torch.argwhere(~ds_tokens['instructed_to_lie']))
# ds_tokens_truthful

In [ ]:
from lie_elicitation_prompts.helpers.torch_helpers import clear_mem
clear_mem()

In [ ]:
from torch.utils.data import DataLoader
from lie_elicitation_prompts.helpers.select import select_multi_from_tensor
from lie_elicitation_prompts.helpers.scores import sum_select_choices_from_logits

batch_size = 10

ds = ds_tokens.select_columns(['ds_string', 'sys_instr_name', 'example_i', 'instructed_to_lie', 'label_true', 'input_ids', 'attention_mask', 'choice_ids'])
dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

model.eval()

results = []

for nb, batch in enumerate(tqdm(dl)):

    # to device
    inputs = {'input_ids': batch['input_ids'].to(model.device), 'attention_mask': batch['attention_mask'].to(model.device)}
    labels = batch['label_true']
    choice_ids = batch['choice_ids']#.to(model.device)

    with torch.no_grad():
        out = model(**inputs)

        # see how elk handles this https://github.com/EleutherAI/elk/blob/84e99a36a5050881d85f1510a2486ce46ac1f942/elk/extraction/extraction.py#L388
        logits_last = out['logits'][:, -1].detach().cpu()
        probs = out['prob_choices'] = sum_select_choices_from_logits(logits_last, choice_ids) # this does not add to one, as it is the prob from among all tokens
        out['coverage'] = probs.sum(dim=1)

        # select the answer
        out['prob_ans'] = prob_ans = select_multi_from_tensor(probs, labels) 
        # ind = torch.arange(labels.size(0))
        # out['prob_ans'] = prob_ans = probs[ind, labels*1]
        out['odds_ans'] = prob_ans / probs.sum(-1) # ratio of probability mass assigned to the true label

        # if we told it to lie, flip the truth odds. we want the odds over the other answer
        instructed_to_lie = batch['instructed_to_lie'] * 1
        out['odds_ans'] = (1-out['odds_ans']) * instructed_to_lie + out['odds_ans'] * (1-instructed_to_lie)

        corrects = out['odds_ans']>0.5

        # FIXME, make my logic forward compatible with multiple chocies, not bool

        for batch_i, correct in enumerate(corrects):
            results.append({
                'instructed_to_lie': batch['instructed_to_lie'][batch_i].item(),
                'ds_string': batch['ds_string'][batch_i],
                'sys_instr_name': batch['sys_instr_name'][batch_i],
                'example_i': batch['example_i'][batch_i].item(),
                'correct': correct.item(),
                'prob_ans': out['prob_ans'][batch_i].item(),
                'odds_ans': out['odds_ans'][batch_i].item(),
                'coverage': out['coverage'][batch_i].item(),
                'prob_choices': out['prob_choices'][batch_i].tolist(),
            })

In [ ]:
# work out which question it knows the answer to
df_results = pd.DataFrame(results)

# # aggregate by original question content
# df_correct_agg = df_results.groupby(['ds_string', 'example_i'])['correct'].agg(['count','mean']).reset_index()
# df_correct_agg

# # we should get all right, and have had at least 2
# df_known = df_correct_agg.query("mean > 0.9 & count > 1")
# print(f'{len(df_correct_agg.query("mean > 0.9"))/len(df_correct_agg):.2%} examples correct')
# print(f'{len(df_correct_agg.query("count > 1"))/len(df_correct_agg):.2%} examples represented')
# print(f'ds len {len(df_correct_agg)}->{len(df_known)}')
# df_known

In [ ]:
# df_results.groupby?

In [ ]:
# for half the dataset it is asked to tell the truth, lets get the question id's where it reliably succeeds. These are places the model knows the truth.
df_ans = (df_results
            .query("instructed_to_lie==False")
            .groupby(['ds_string', 'example_i'], as_index=0)['correct'].agg(['count','mean'])
)
df_known = (df_ans
            .query("mean > 0.9 & count > 1")
            .drop(columns=['count','mean'])
)
print(f'{len(df_known)/len(df_ans):.2%} of the time the model got the questions reliably correct')
df_known

In [ ]:
# # QC
# # print(df_results.groupby(['ds_string', 'example_i'])['instructed_to_lie'].mean())

# # how often was it correct, when asked to lie
# df_results.groupby(['instructed_to_lie'])['correct'].mean()

In [ ]:
# also look at the half where it was asked to lie, and find where it reliably lies
df_lie_res_agg = (df_results
            .query("instructed_to_lie==True")
            .groupby(['ds_string', 'example_i'], as_index=0)['correct'].agg(['count','mean'])
)
df_lies = (df_lie_res_agg
            .query("mean > 0.9 & count > 1")
            .drop(columns=['count','mean'])
)
print(f'{len(df_lies)/len(df_lie_res_agg):.2%} of the time the model reliably lied when asked')

In [ ]:
df_results

In [ ]:
# QC lies by ds
df_lie_res_agg = (df_results
            .query("instructed_to_lie==True")
            .groupby(['ds_string'], as_index=0)['correct'].agg(['count','mean'])
)
df_lie_res_agg

In [ ]:
# QC lies by prompt
df_lie_res_agg = (df_results
            .query("instructed_to_lie==True")
            .groupby(['sys_intr'], as_index=0)['correct'].agg(['count','mean'])
)
df_lie_res_agg

In [ ]:
# find our lies dataset
pd.merge(df_known, df_lies)

In [ ]:
# QC

# On a good dataset: Acc, or prob on correct ans should be high
# And on a well formatted dataset, coverage should be hihg
df_results.groupby(['ds_string'])[['coverage', 'odds_ans']].mean()

In [ ]:
def row_is_known(x):
    k = df_known[df_known.ds_string==x['ds_string']]
    return x['example_i'].item() in k.example_i.values

# filter the dataset to known answers based on ds_string and example_i
ds_tokens_known = ds_tokens.filter(row_is_known)
print(f"{len(ds_tokens)} -> {len(ds_tokens_known)}")
ds_tokens_known

In [ ]:
(ds_tokens_known['instructed_to_lie']*1.0).mean()

In [ ]:
# save
ts = pd.Timestamp.now().strftime('%Y%m%d-%H%M%S')
f = Path(f"../data/extracted_prompts_{ts}")
print(f)
ds_tokens_known.info.description = json.dumps(cfg.__dict__)
ds_tokens_known.save_to_disk(str(f))

In [ ]:
# push to hf https://huggingface.co/docs/datasets/v2.20.0/en/package_reference/main_classes#datasets.Dataset.push_to_hub
ds_tokens_known.push_to_hub('wassname/abliterated-llama-known-prompts', split='train', config_name='')

In [ ]:
# TODO see if it will also lie on an answer...

## QC

In [ ]:
# # which source datasets did the known questions come from?
# df_ds = ds_tokens_known.to_pandas()
# df_ds[['ds_string','sys_instr_name']].value_counts()

In [ ]:
# df_metadata = ds_tokens.select_columns(['ds_string', 'sys_instr_name', 'answer_choices', 'label_true', 'instructed_to_lie']).to_pandas()

In [ ]:
# # QC a batch
# ss = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=False)
# for s in ss:
#     s = s.replace(tokenizer.eos_token, '')
#     s = s.replace('<|start_header_id|>', '\n[')
#     s = s.replace('<|end_header_id|>', ']')
#     tokenizer.chat_template
#     print('---')
#     print(s)